<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/llm_makes_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Import packages

import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from datetime import datetime

In [ ]:
# Set base parameters

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/Scrapes"
LOAD_FILE = os.path.join(LOAD_PATH, "guideline_raw.json")

SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "guideline_structured.json")

# Error path and file name are defined below in orchestrate function to allow time-stamp

In [ ]:
# Load LLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto")

In [ ]:
# Check GPU connection

import torch
print(torch.cuda.is_available())
print(model.device)

True
cuda:0


In [ ]:
# Test

## Should also carry out test prompt of transforming recommendations?

text = "Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable?"

inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=500)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable? No, not necessarily. Lithium is a mood stabilizer commonly used in the treatment of bipolar disorder, and it can help manage symptoms such as mania, hypomania, depression, and irritability. Mild irritability alone does not typically warrant a referral to secondary care.

However, it's important to monitor the individual's overall mental health status and any changes in their condition. If the irritability persists or worsens, or if there are other concerning symptoms (such as significant mood swings, suicidal thoughts, psychotic features, or side effects from the medication), then a referral to secondary care might be appropriate for a more comprehensive evaluation and management plan.

Regular follow-ups with a primary care provider or a psychiatrist are crucial to ensure that the treatment plan is effective and safe. They can adjust the medication dosage, add o

In [ ]:
# Load JSON of raw recommendations

def load_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {file_path}')

raw_recommendations = load_json(LOAD_FILE)

print(type(raw_recommendations))
print(len(raw_recommendations))
print(raw_recommendations[0])

<class 'list'>
136
{'heading_1': '1.1 Care for adults, children and young people across all phases of bipolar disorder', 'sub_heading_1': 'Treatment and support for specific populations', 'sub_heading_2': None, 'original_recommendation_number': '1.1.1', 'original_recommendation_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder.'}


In [ ]:
def construct_prompt(entity):

    # Need to do manual read-through of entire guideline to check value fidelity
    # Risk: maybe risk_to_others hasn't been populated at all, is that correct?
    # Work on physical health fields to increase specificity?
    # current_manic_phase is only populating occasionally with "mania" and not other values: incorrect?

    """
    Given one recommendation entry {}, creates the prompt to extract one normalised JSON item
    """

    heading_1 = entity.get('heading_1')
    sub_heading_1 = entity.get('sub_heading_1')
    sub_heading_2 = entity.get('sub_heading_2')

    original_recommendation_number = entity.get('original_recommendation_number')
    original_recommendation_text = entity.get('original_recommendation_text')

    heading_context = " > ".join(h.strip() for h in [heading_1, sub_heading_1, sub_heading_2] if isinstance(h, str) and h.strip())

    return f"""
    You are extracting structured information from a NICE guideline recommendation.

    RULES:
    - output must be valid JSON ONLY (no markdown; no commentary)
    - do not invent clinical information, thresholds or populations; use only what is present in the recommendation text
    - For ALL clinical descriptor fields (e.g. phase, severity, medication): populate a value ONLY if it is explicitly stated in the recommendation text. Do NOT infer information that is not directly stated. If not explicit, use null.
    - 'conditionality' must be a string describing the constraint or constraints on the implementation of a recommended clinical action, indicated by clauses such as 'if...', 'where...', 'when...' or 'who are...' (e.g., 'if a person has rapid or excessive weight gain, abnormal lipid levels or problems with blood glucose management'). If none is mentioned, record null. If multiple conditions, return a list of strings.
    - 'action' must be a verb phrase indicating the clinical decision or intervention (e.g., continue treatment, give advice...) that is being done to/for a person. It should capture the clinical instruction and not merely the verb (e.g., 'ensure that people have access to calming environments and reduced stimulation', not just 'ensure'). If more than one action is mentioned, retain all as a list of strings. Do NOT include information giving (indicated by 'explain', 'discuss' or similar verbs): This should be stored in 'information_giving'.
    - 'information_giving' must be a verb phrase indicating sharing information, explaining or discussing with the patient. If not mentioned, set to null.
    - Extract 'prohibitions_and_cautions' from verb phrases including 'do not', 'must not' or 'should not', and clauses beginning 'but...'. If none are mentioned, record null.
    - 'population' is the patient or staff group that the recommendation applies to (e.g., 'people with bipolar…', 'pregnant women…', 'psychological therapists', etc.). If none is mentioned, record null.
    - 'age_group' must be one of: ['child', 'young_person', 'older_adult', 'adult', null]
    - Extract 'urgency' as True if the text includes 'urgent', 'urgently', 'immediate' or 'immediately', otherwise False
    - 'manic_episode_history' must be one of: ['none', 'one', 'multiple', null]
    - 'current_manic_phase' must be one of: ['mania', 'hypomania', 'depression', 'mixed', 'rapid_cycling', 'euthymic', null]
    - 'symptom_severity' must be one of: ['mild', 'moderate', 'severe', null]
    - 'current_psychosis' must be: ['present', 'absent', null]
    - 'diagnoses' must be one or more mental health diagnoses. If more than one diagnosis is mentioned, record all as a list of strings. If none are mentioned, record null.
    - 'current_medication' should be populated by medication names if the recommendation states explicitly that the patient is taking them currently. Otherwise set it to null and record any medications in 'suggested_medications'. If there are multiple medication names, retain each as a list of strings.
    - 'medication_adherence' must be one of: ['good', 'poor', null]
    - 'suggested_medication' should be populated by medication names that are suggested as a treatment action, or set to null. If there are multiple medication names, retain each as a list of strings. If the medications are to be taken together (as suggested by 'combined'), record as a single string.
    - 'physical_health_longterm' must be the name of a physical disease diagnosis that usually affects a person for more than six months. If more than one diagnosis is mentioned, record all as a list of strings. If none are mentioned, record null.
    - 'physical_health_recent' must be the name of a transient disease (less than six months) or physical health event from the last six months. If more than one diagnosis is mentioned, record all as a list of strings. If none are mentioned, record null.
    - 'risk' must be one of: ['self_harm', 'risk_to_others', null]
    - If 'psychological_therapy' is mentioned in the recommendation, set to True, otherwise False.
    - 'other_treatments' should be populated by any treatment name that is not a medication or psychological therapy, or set to null.
    - 'clinical_setting' must be one of: ['assessment', 'primary_care', 'secondary_care', 'inpatient', null]
    - 'care_coordination' must be one of: ['current', 'offered', null]

    CONTEXT: {heading_context}

    RECOMMENDATION NUMBER: {original_recommendation_number}
    RECOMMENDATION TEXT: {original_recommendation_text}

    Produce JSON with exactly these keys:
    - heading_context
    - original_recommendation_number
    - original_recommendation_text
    - conditionality
    - action
    - information_giving
    - prohibitions_and_cautions
    - population
    - age_group
    - urgency
    - manic_episode_history
    - current_manic_phase
    - symptom_severity
    - current_psychosis
    - diagnoses
    - current_medication
    - medication_adherence
    - suggested_medication
    - physical_health_longterm
    - physical_health_recent
    - risk
    - psychological_therapy
    - other_treatments
    - clinical_setting
    - care_coordination

    REMEMBER:
    - you MUST use null if the information for any field is not explicit in the recommendation or heading
    - if there is more than one value for any field, retain all as a list of strings
    """

In [ ]:
# Check prompt length

recommendation_for_prompt_check = raw_recommendations[24]

prompt_test = construct_prompt(recommendation_for_prompt_check)

token_count = tokenizer.encode(prompt_test)

print(f"Prompt token length: {len(token_count)}\n")

print(f"Recommendation used for prompt check: {recommendation_for_prompt_check}")

Prompt token length: 1448

Recommendation used for prompt check: {'heading_1': '1.3 Assessing suspected bipolar disorder in adults in secondary care', 'sub_heading_1': None, 'sub_heading_2': None, 'original_recommendation_number': '1.3.2', 'original_recommendation_text': "When assessing suspected bipolar disorder:, and undertake a full psychiatric assessment, documenting a detailed history of mood, episodes of overactivity and disinhibition or other episodic and sustained changes in behaviour, symptoms between episodes, triggers to previous episodes and patterns of relapse, and family history, and assess the development and changing nature of the mood disorder and associated clinical problems throughout the person's life (for example, early childhood trauma, developmental disorder or cognitive dysfunction in later life), and assess social and personal functioning and current psychosocial stressors, and assess for potential mental and physical comorbidities, and assess the person's phys

In [ ]:
def run_llm_on_entity(tokenizer, model, entity):

    """
    Call the model on a single prompt using the prompt function
    Return model response
    """

    prompt = construct_prompt(entity)

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    llm_response = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return llm_response[0]

Source for nested curly braces extraction: https://til.magmalabs.io/posts/01a278bb48-extracting-json-code-with-nested-curly-braces-in-ruby-the-long-painful-way-around-with-help-from-gpt4

In [ ]:
def convert_output_to_true_json(llm_response):
    """
    Takes output from run_llm_on_entity
    Turns it into a true JSON dictionary
    Checks whether it is a JSON file
    """

    llm_response = llm_response.strip()

    start = llm_response.find("{")

    if start == -1: # Where -1 is not found in .find string method
        raise ValueError("In converting_output_to_true_json function, no initial { was found in the output from the LLM.\n")

    brace_count = 0
    json_string = None

    for i in range(start, len(llm_response)):

        if llm_response[i] == "{":
            brace_count += 1
        elif llm_response[i] == "}":
            brace_count -= 1

            if brace_count == 0:

                json_string = llm_response[start:i + 1].strip()
                break

    if json_string is None:
        raise ValueError("In converting_output_to_true_json function, no closing } was found in the output from the LLM.\n")

    try:
        json_object = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        raise

    if not isinstance(json_object, dict):
        raise TypeError(f"The object created by the function convert_output_to_true_json is not a JSON object. Instead it is a {type(json_object)}.")

    return json_object

In [ ]:
def validate_json(json_object, required_keys):

    """
    Checks the structure of the JSON to see whether it has the required keys
    ... and is populated with the right types of data
    """

    missing_keys = [k for k in required_keys if k not in json_object]
    if missing_keys:
        raise ValueError(f"Missing required keys: {missing_keys}")

    extra_keys = [k for k in json_object.keys() if k not in required_keys]
    if extra_keys:
        raise ValueError(f"Unexpected extra keys: {extra_keys}")

    for key, value in json_object.items():
        if value is None:
            continue

        # Fields with constrained values
        if key == 'age_group' and value in {'child','young_person','older_adult','adult'}:
            continue
        if key == 'clinical_setting' and value in {'assessment', 'primary_care', 'secondary_care', 'inpatient'}:
            continue
        if key == 'manic_episode_history' and value in {'none', 'one', 'multiple'}:
            continue
        if key == 'current_manic_phase' and value in {'mania', 'hypomania', 'depression', 'mixed', 'rapid_cycling', 'euthymic', None}:
            continue
        if key == 'symptom_severity' and value in {'mild', 'moderate', 'severe'}:
            continue
        if key == 'current_psychosis' and value in {'present', 'absent'}:
            continue
        if key == 'medication_adherence' and value in {'good', 'poor'}:
            continue
        if key == 'risk' and value in {'self_harm', 'risk_to_others'}:
            continue
        if key == 'care_coordination' and value in {'current', 'offered'}:
            continue

        # Boolean fields
        if key in {"urgency", "psychological_therapy"} and isinstance(value, bool):
            continue

        # String or list of string fields
        if isinstance(value, str):
            continue

        if isinstance(value, list) and all(isinstance(v, str) for v in value):
            continue

        raise TypeError(f"Key '{key}' is of the wrong type, namely: {type(value)}.")

    return json_object

In [ ]:
def orchestrate_create_json(raw_recommendations, tokenizer, model, save_file):

    compiled_recommendations = []

    errors = []

    required_keys = ['heading_context',
                    'original_recommendation_number',
                    'original_recommendation_text',
                    'action',
                    'information_giving',
                    'prohibitions_and_cautions',
                    'population',
                    'age_group',
                    'conditionality',
                    'urgency',
                    'manic_episode_history',
                    'current_manic_phase',
                    'symptom_severity',
                    'current_psychosis',
                    'diagnoses',
                    'current_medication',
                    'suggested_medication',
                    'medication_adherence',
                    'physical_health_longterm',
                    'physical_health_recent',
                    'risk',
                    'psychological_therapy',
                    'other_treatments',
                    'clinical_setting',
                    'care_coordination']

    counter = 0

    for i, entity in enumerate(raw_recommendations): # Add [:] slicing for test runs

        llm_output_text = run_llm_on_entity(tokenizer, model, entity)

        try:
            parsed_json = convert_output_to_true_json(llm_output_text)
            validate_json(parsed_json, required_keys)

        except Exception as e:
            print(f"Error {e} at point {i}")
            errors.append({"index": i, "error": str(e), "raw_llm_output": llm_output_text, "entity": entity})
            continue

        compiled_recommendations.append(parsed_json)

        counter += 1

        if counter % 10 == 0: print(f'Number of recommendations processed: {counter}')

    with open(save_file, "w", encoding="utf-8") as f:
        json.dump(compiled_recommendations, f, ensure_ascii=False, indent=2)

    datetime_now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    ERROR_SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON/LLM_conversion_errors"
    os.makedirs(ERROR_SAVE_PATH, exist_ok=True)
    ERROR_SAVE_FILE = os.path.join(ERROR_SAVE_PATH, f"llm_conversion_errors_{datetime_now}.json")

    with open(ERROR_SAVE_FILE, "w", encoding="utf-8") as f:
        json.dump(errors, f, ensure_ascii=False, indent=2)

    print(f"Here is the list of json parsing errors: {errors}\n\n")

    return compiled_recommendations, errors

In [ ]:
orchestrate_create_json(raw_recommendations, tokenizer, model, SAVE_FILE)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Error unhashable type: 'list' at point 3
Error unhashable type: 'list' at point 4
Error unhashable type: 'list' at point 8
Error unhashable type: 'list' at point 10
Error In converting_output_to_true_json function, no closing } was found in the output from the LLM.
 at point 13
Number of recommendations processed: 10
Error In converting_output_to_true_json function, no closing } was found in the output from the LLM.
 at point 24
Number of recommendations processed: 20
Error unhashable type: 'list' at point 27
Error In converting_output_to_true_json function, no closing } was found in the output from the LLM.
 at point 28
Error unhashable type: 'list' at point 30
Error unhashable type: 'list' at point 31
Error unhashable type: 'list' at point 32
Error unhashable type: 'list' at point 36
Error unhashable type: 'list' at point 39
Number of recommendations processed: 30
Error In converting_output_to_true_json function, no closing } was found in the output from the LLM.
 at point 47
Error I

([{'heading_context': '1.1 Care for adults, children and young people across all phases of bipolar disorder > Treatment and support for specific populations',
   'original_recommendation_number': '1.1.1',
   'original_recommendation_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder.',
   'conditionality': None,
   'action': ['offer the same range of treatments and services'],
   'information_giving': None,
   'clinical_setting': None,
   'population': 'older people with bipolar disorder',
   'age_group': 'older_adult',
   'prohibitions_and_cautions': None,
   'urgency': False,
   'manic_episode_history': None,
   'current_manic_phase': None,
   'symptom_severity': None,
   'current_psychosis': None,
   'diagnoses': ['bipolar disorder'],
   'current_medication': None,
   'medication_adherence': None,
   'suggested_medication': None,
   'physical_health_longterm': None,
   'physical_health_

In [ ]:
# For checking what caused specific JSON parsing errors

parsing_error_line = 23
parsing_error_column = str(1)
parsing_error_character = 728

guideline_structured_error_location = load_json(SAVE_FILE)

slice_ = guideline_structured_error_location[parsing_error_line][parsing_error_column]
exact_character = slice_[728]

print(f"The offending entry: {slice_}\n")
print(f"The offending character: {exact_character}")